In [ ]:
# Choose Google T4 GPU to run
!nvidia-smi

In [ ]:
# Get the latest update of Ubuntu Runtime
!sudo apt-get update

# Install poppler-utils in PATH
!sudo apt-get install poppler-utils

# Install modules for pdf conversion
!pip install pdf2image

In [ ]:
# Link to Google Drive if you want persistent storage
# from google.colab import drive
# drive.mount('/content/drive')
# %cd "/content/drive/MyDrive/Colab Notebooks"

In [ ]:
# Create all necessary folders
%mkdir INPUT_PDF OUTPUT_JPG IMAGE_BEFORE IMAGE_AFTER
%ls -ls

# Section A: Conversion of PDF to JPEG

In [ ]:
import os
from pdf2image import convert_from_path

In [ ]:
PDF_LOCATION = './INPUT_PDF'
JPG_LOCATION = './OUTPUT_JPG'

source_dir_exist = os.path.exists(PDF_LOCATION)
dest_dir_exist = os.path.exists(JPG_LOCATION)

if not source_dir_exist:
	print(f'The PDF input storage folder {PDF_LOCATION} does not exist. Create a new folder for you.')
	os.makedirs(PDF_LOCATION)

if not source_dir_exist:
	print(f'The JPG output storage folder {JPG_LOCATION} does not exist. Create a new folder for you.')
	os.makedirs(JPG_LOCATION)

print(f'Folders {PDF_LOCATION} and {JPG_LOCATION} are ready. Please place the pdf files under {PDF_LOCATION}')

In [ ]:
DEFAULT_PAGE = 1
file_doc_list = os.listdir(PDF_LOCATION)
print(f'Currently the folder {PDF_LOCATION} contains {len(file_doc_list)} files.')
print(f'Please place pdf files that you want to convert under {PDF_LOCATION}.')
file_doc_list.sort()

for doc_x in file_doc_list:
	if doc_x.endswith("pdf"):
		pages = convert_from_path(os.path.join(PDF_LOCATION, doc_x))
		# DEFAULT_PAGE = len(pages)
		print(f'The PDF file {doc_x} contains {len(pages)} page(s). Only the first {DEFAULT_PAGE} will be extracted.')
		for i in range(0, DEFAULT_PAGE):
			doc_name = doc_x.split('.')[0] + "_" + str(i+1) + ".jpg"
			dest_path = os.path.join(JPG_LOCATION, doc_name)
			pages[i].save(dest_path)
			print(f'The {i+1} page of the PDF document {doc_x} has been coverted to {doc_name} and saved at {dest_path}')

print(f'The conversion has completed.')

In [ ]:
# Zip the outputs as outputs.zip. Refresh and download the zip file since data is not persistent.
!zip -r im_outputs.zip ./OUTPUT_JPG

# Section B: Image augmentation

In [ ]:
# Use this command for
# Fully copy of images from OUTPUT_JPG -> IMAGE_BEFORE
# !cp ./OUTPUT_JPG/*.*  ./IMAGE_BEFORE
%pwd

In [ ]:
import cv2
import os

In [ ]:
IMAGE_BEFORE_LOCATION = './IMAGE_BEFORE'
IMAGE_AFTER_LOCATION = './IMAGE_AFTER'

# Check existence of folders under current directory
before_dir_exist = os.path.exists(IMAGE_BEFORE_LOCATION)
after_dir_exist = os.path.exists(IMAGE_AFTER_LOCATION)

if not before_dir_exist:
	print(f'The JPG / PNG input storage folder {IMAGE_BEFORE_LOCATION} does not exist. Create a new folder for you.')
	os.makedirs(IMAGE_BEFORE_LOCATION)
if not after_dir_exist:
	print(f'The augmented JPG / PNG output storage folder {IMAGE_AFTER_LOCATION} does not exist. Create a new folder for you.')
	os.makedirs(IMAGE_AFTER_LOCATION)
print(f'Folders {IMAGE_BEFORE_LOCATION} and {IMAGE_AFTER_LOCATION} are ready. Please place the JPG / PNG image files under {IMAGE_BEFORE_LOCATION}')


In [ ]:
im_list = os.listdir(IMAGE_BEFORE_LOCATION)
im_list.sort()
print(f'Currently the folder {IMAGE_BEFORE_LOCATION} contains {len(im_list)} files.')
print(f'Please place the JPG / PNG image files that you want to convert under {IMAGE_BEFORE_LOCATION}.')


def augmentation_task(im):
	im_location = os.path.join(IMAGE_BEFORE_LOCATION, im)
	im_details = cv2.imread(im_location)

	scale_percent = 50
	width = int(im_details.shape[1] * scale_percent / 100)
	height = int(im_details.shape[0] * scale_percent / 100)
	im_dim = (width, height)
	action_dict = {
		0: cv2.GaussianBlur(im_details, (5, 5), 0),
		1: cv2.GaussianBlur(im_details, (7, 7), 0),
		2: cv2.GaussianBlur(im_details, (9, 9), 0),
		3: cv2.cvtColor(im_details, cv2.COLOR_BGR2GRAY),
		4: cv2.resize(im_details, im_dim, interpolation = cv2.INTER_AREA)
	}
	action_str = {0:"blur1", 1:"blur2", 2:"blur3",3:"gray", 4:"resize"}
	key_list = list(action_dict.keys())

	for num in key_list:
		new_im_name = im.split(".")[0] + "_" + action_str[num] + "." + im.split(".")[-1]
		final_path = os.path.join(IMAGE_AFTER_LOCATION, new_im_name)
		cv2.imwrite(final_path, action_dict[num])
		print(f'The augmented image of {im} : {new_im_name} is created and saved at {final_path}')

def rotation_task(im):
	im_location = os.path.join(IMAGE_AFTER_LOCATION, im)
	im_details = cv2.imread(im_location)

	rotation_action_dict = {
	0: cv2.rotate(im_details, cv2.ROTATE_90_CLOCKWISE),
	1: cv2.rotate(im_details, cv2.ROTATE_180),
	2: cv2.rotate(im_details, cv2.ROTATE_90_COUNTERCLOCKWISE)
	}
	rotate_str = {0: '090', 1:'180', 2:'270'}
	rotation_key_list = list(rotation_action_dict.keys())

	for num in rotation_key_list:
		new_im_name = im.split(".")[0] + "_" + rotate_str[num] + "." + im.split(".")[-1]
		final_path = os.path.join(IMAGE_AFTER_LOCATION, new_im_name)
		cv2.imwrite(final_path, rotation_action_dict[num])
		print(f'The rotated image of {im} : {new_im_name} is created and saved at {final_path}')

for im in im_list:
	if im.endswith('.jpg') or im.endswith('.png'):
		augmentation_task(im)

rotate_choice = input('Rotate the augmented images as well? (Y/N): ')

if rotate_choice == 'Y':
	im_after_list = os.listdir(IMAGE_AFTER_LOCATION)
	im_after_list.sort()

	for im in im_after_list:
		if im.endswith('.jpg') or im.endswith('.png'):
			rotation_task(im)

In [ ]:
# Zip the outputs as outputs.zip. Refresh and download the zip file since data is not persistent.
!zip -r aug_outputs.zip ./IMAGE_AFTER